<a href="https://colab.research.google.com/github/aritrartira/Football-Statistics-Visualisations/blob/add-license-1/src/XPointsHome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
LEAGUE = 'epl'
SEASON = '2019'     # Example - For 2019-20 use '2019'
TEAM = 'Manchester United'
FILENAME = f"xPoints_MUN_home_games_{SEASON}_30_3_2021.jpg"

# Installing Necessary Libraries

In [ ]:
!pip install aiohttp
!pip install understat

# Importing Necessary Libraries

In [ ]:
import asyncio
import json
import numpy as np

import aiohttp

from understat import Understat

import nest_asyncio
nest_asyncio.apply()

# Code

In [ ]:
async def main():
    global teams
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        teams = await understat.get_teams(LEAGUE, SEASON)
        print(json.dumps(teams))

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

id_dict = dict()

for team in teams:
    id_dict[team['title']] = team['id']

id_dict

In [ ]:
short_title_dict = dict()

for team in id_dict.keys():
    async def main():
        global results
        async with aiohttp.ClientSession() as session:
            understat = Understat(session)
            short_title_dict[team] = (await understat.get_team_results(team, SEASON, side="h"))[0]['h']['short_title']

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

short_title_dict

In [ ]:
stats = 0
async def main():
    global stats
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        stats = await understat.get_league_results(LEAGUE, SEASON, {'h' : {'id' : id_dict[TEAM], 'title' : TEAM,  "short_title": short_title_dict[TEAM]}})
        #stats.append(await understat.get_league_results('EPL', '2018', {'a' : {'id' : '89', 'title' : 'Manchester United',  "short_title": "MUN"}}))

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [ ]:
match_ids = []

for match in stats:
    match_ids.append(int(match['id']))

match_ids

In [ ]:
match_shots = []

for id in match_ids:
    async def main():
        global match_shots
        async with aiohttp.ClientSession() as session:
            understat = Understat(session)
            match_shots.append(await understat.get_match_shots(id))

    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


len(match_shots)

In [ ]:
for i in range(len(match_shots)):
    match = match_shots[i]
    a = []
    h = []
    for shot in match['a']:
        a.append(float(shot['xG']))

    for shot in match['h']:
        h.append(float(shot['xG']))

    match_shots[i] = a, h

match_shots

In [ ]:
import random

outcomes = dict()

def predict_goals(xG):
    goals = 0

    for i in xG:
        if (random.random() <= i):
            goals = goals + 1
    
    return goals

def simulate(A_xG, B_xG):
    for i in range(100000):
        goals_A = predict_goals(A_xG)
        goals_B = predict_goals(B_xG)

        outcome = (str(goals_A), str(goals_B))

        if outcome in outcomes.keys():
            outcomes[outcome] = outcomes[outcome] + 1
        else:
            outcomes[outcome] = 1

def reset_outcomes():
    global outcomes
    outcomes = dict()


def prob_A_win(A_xG, B_xG):
    reset_outcomes()
    simulate(A_xG, B_xG)
    
    fav_outcomes = 0
    for i in outcomes.keys():
        if int(i[0]) > int(i[1]):
            fav_outcomes = fav_outcomes + outcomes[i]

    prob = fav_outcomes / 100000.0
    return prob

def prob_B_win(A_xG, B_xG):
    reset_outcomes()
    simulate(A_xG, B_xG)
    
    fav_outcomes = 0
    for i in outcomes.keys():
        if int(i[0]) < int(i[1]):
            fav_outcomes = fav_outcomes + outcomes[i]

    prob = fav_outcomes / 100000.0
    return prob

def prob_draw(A_xG, B_xG):
    reset_outcomes()
    simulate(A_xG, B_xG)
    
    fav_outcomes = 0
    for i in outcomes.keys():
        if int(i[0]) == int(i[1]):
            fav_outcomes = fav_outcomes + outcomes[i]

    prob = fav_outcomes / 100000.0
    return prob

def most_likely_outcome(A_xG, B_xG):
    reset_outcomes()
    simulate(A_xG, B_xG)
    most_likely = max(outcomes, key = outcomes.get)

    return str(most_likely[0]) + "-" + str(most_likely[1])

In [ ]:
xPoints = []

for match in match_shots:
    prob_w = prob_B_win(match[0], match[1])
    prob_d = prob_draw(match[0], match[1])
    xPoints.append(prob_w * 3 + prob_d)

xPoints

In [ ]:
import numpy as np

Y = xPoints

p = np.polyfit([i for i in range(1, len(Y) + 1)], Y, 1)

Y_line = [p[0] * x + p[1] for x in range(1, len(Y) + 1)]

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot([i for i in range(1, len(Y) + 1)], Y, label = f'xPoints for {short_title_dict[TEAM]} Home Games {SEASON}-{int(SEASON[-2:]) + 1}')
plt.plot([i for i in range(1, len(Y) + 1)], Y_line)
plt.scatter([i for i in range(1, len(Y) + 1)], Y)
#plt.xticks(ticks = [i for i in range(1, 21)], labels = X)
#plt.plot([i for i in range(1, len(Y) + 1)], [np.mean(np.array(Y)) for _ in range(1, len(Y) + 1)], 'r--', label = "Average xA of all key passes")
#plt.axhspan(np.mean(np.array(Y)), 0.6, xmin=0, xmax=21, alpha = 0.2, facecolor = 'green', label = "Overperforming the average")
#plt.axhspan(np.mean(np.array(Y)), 0, xmin=0, xmax=21, alpha = 0.2, facecolor = 'red', label = "Underperforming the average")
plt.legend()
plt.ylabel('xPoints')
plt.xlabel('Matches')
plt.savefig(FILENAME, dpi = 200)
plt.show()